In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
default_fontsize = plt.rcParams['font.size']
publication_fontsize_large = 20
publication = True
if publication: plt.rcParams.update({'font.size': publication_fontsize_large})

## Contact Space for a Molecular System (0D)

### Generate the Molecule

In [ ]:
from ase.build import molecule

In [ ]:
water = molecule('H2O')
water.translate([5, 5, 5])
water.set_cell([10, 10, 10])

In [ ]:
water.positions

In [ ]:
from mapsy.data import Grid
water_grid: Grid = Grid(cell=water.cell)

In [ ]:
from mapsy.data import System
system_0D: System = System(water_grid, water, dimension=0)

### Contact Space on System Boundary

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "system", "distance": 2.5, "spread": 0.5, "cutoff": 100, "threshold": 0.5, 'side':1})
system_contactspace = ContactSpaceGenerator(contactspacesettings).generate(system_0D)

In [ ]:
xxx,yyy,zzz = system_contactspace.boundary.switch.tocontour([5,5,5],0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.01,0.2,0.4,0.6,0.8,0.99],colors='black')
plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*water.positions[0,1:].T,c='r',edgecolor='black',s=200)
plt.scatter(*water.positions[1:,1:].T,c='w',edgecolor='black',s=80)

plt.title(r'$s(r)$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Y (Å)')
plt.xticks(list(range(0, 11, 2)))
plt.yticks(list(range(0, 11, 2)))
plt.axis('square')
plt.xlim(0,10)
plt.ylim(0,10)
plt.show()

In [ ]:
xxx,yyy,zzz = system_contactspace.boundary.gradient.modulus.tocontour([5,5,5],0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.1,0.4,0.8,1.2],colors='black')
#plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*water.positions[0,1:].T,c='r',edgecolor='black',s=200)
plt.scatter(*water.positions[1:,1:].T,c='w',edgecolor='black',s=80)

filter_data = system_contactspace.data[np.abs(system_contactspace.data['x'].values-5) < 0.01]
plt.scatter(filter_data[['y']].values,filter_data[['z']].values,s=1,alpha=0.5)

plt.title(r'$\left|\nabla s(r)\right|$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Y (Å)')
plt.xticks(list(range(0, 11, 2)))
plt.yticks(list(range(0, 11, 2)))
plt.axis('square')
plt.xlim(0,10)
plt.ylim(0,10)
plt.show()

### Contact Space on Ionic Boundary (Soft-Spheres)

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "ionic", "radii": "muff", "alpha":1.12, "spread": 0.5, "cutoff": 100, "threshold": 0.4, 'side':1})
ionic_contactspace = ContactSpaceGenerator(contactspacesettings).generate(system_0D)

In [ ]:
xxx,yyy,zzz = ionic_contactspace.boundary.switch.tocontour([5,5,5],0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.01,0.2,0.4,0.6,0.8,0.99],colors='black')
plt.contourf(xxx,yyy,zzz,levels=100,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*water.positions[0,1:].T,c='r',edgecolor='black',s=200)
plt.scatter(*water.positions[1:,1:].T,c='w',edgecolor='black',s=80)

plt.title(r'$s(r)$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Y (Å)')
plt.xticks(list(range(0, 11, 2)))
plt.yticks(list(range(0, 11, 2)))
plt.axis('square')
plt.xlim(0,10)
plt.ylim(0,10)
plt.show()

In [ ]:
xxx,yyy,zzz = ionic_contactspace.boundary.gradient.modulus.tocontour([5,5,5],0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.1,0.4,0.8],colors='black')
#plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*water.positions[0,1:].T,c='r',edgecolor='black',s=200)
plt.scatter(*water.positions[1:,1:].T,c='w',edgecolor='black',s=80)

filter_data = ionic_contactspace.data[np.abs(ionic_contactspace.data['x'].values-5) < 0.01]
plt.scatter(filter_data[['y']].values,filter_data[['z']].values,s=1,alpha=0.8)

plt.title(r'$\left|\nabla s(r)\right|$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Y (Å)')
plt.xticks(list(range(0, 11, 2)))
plt.yticks(list(range(0, 11, 2)))
plt.axis('square')
plt.xlim(0,10)
plt.ylim(0,10)
plt.show()

## Test Calculation of Grid Spacing

In [ ]:
from mapsy.utils.constants import BOHR_RADIUS_ANGS
from mapsy.utils import setscalars
cutoff = 100 # reciprocal space cutoff in Ry (same convetion as QE)
scalars = setscalars(system_0D.grid.cell,cutoff)
print(scalars)
print(water_grid.cell/scalars)

## Test van der Waals Radii

In [ ]:
from mapsy.utils import get_vdw_radii

In [ ]:
atomic_numbers = system_0D.atoms.get_atomic_numbers()
for atomic_number in atomic_numbers:
    print(atomic_number,get_vdw_radii(atomic_number,'uff'))

## Contact Space for a Planar System (2D)

### Generate a Pt(111) Slab

In [ ]:
from ase.build import fcc111
Pt111 = fcc111("Pt",size=(4,4,3),a=3.94,orthogonal=True,periodic=True,vacuum = 5)
Pt111.translate([3.94/8,0,0])

In [ ]:
from mapsy.data import Grid
Pt111_grid: Grid = Grid(cell=Pt111.cell)

In [ ]:
from mapsy.data import System
system_2D: System = System(Pt111_grid, Pt111, dimension=2, axis=2)

### Build System Boundary

In [ ]:
from mapsy.utils import setscalars
cutoff = 100 # reciprocal space cutoff in Ry (same convetion as QE)
scalars = setscalars(system_2D.grid.cell,cutoff)
contact_space_grid = Grid(scalars=scalars,cell=system_2D.grid.cell)

In [ ]:
from mapsy.boundaries import SystemBoundary
system_boundary = SystemBoundary(mode = 'system', grid=contact_space_grid, distance = 4, spread = 1, system = system_2D, label='test')
system_boundary.update()

In [ ]:
xxx,yyy,zzz = system_boundary.switch.tocontour(system_2D.atoms.get_center_of_mass(),1)
contour = plt.contour(xxx,yyy,zzz,levels=[0.01,0.2,0.4,0.6,0.8,0.99],colors='black')
plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*Pt111.positions[:,::2].T,c='grey',edgecolor='black',s=200,alpha=0.2)
filter_Pt111_atoms = abs(Pt111.positions[:,1]-Pt111.get_center_of_mass()[1])<1.
plt.scatter(*Pt111.positions[filter_Pt111_atoms][:,::2].T,c='grey',edgecolor='black',s=200)

plt.title(r'$s(r)$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(system_2D.grid.cell[0,0])+1, 2)))
plt.yticks(list(range(0, int(system_2D.grid.cell[2,2])+1, 2)))
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "system", "distance": 4, "spread": 1, "cutoff": 100, "threshold": 0.4, 'side':1})
system_2D_contactspace_top = ContactSpaceGenerator(contactspacesettings).generate(system_2D)
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "system", "distance": 4, "spread": 1, "cutoff": 100, "threshold": 0.4, 'side':-1})
system_2D_contactspace_bottom = ContactSpaceGenerator(contactspacesettings).generate(system_2D)

In [ ]:
xxx,yyy,zzz = system_boundary.gradient.modulus.tocontour(system_2D.atoms.get_center_of_mass(),1)
contour = plt.contour(xxx,yyy,zzz,levels=[0.1,0.4,0.8,1.2],colors='black')
#
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
#
#plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*Pt111.positions[:,::2].T,c='grey',edgecolor='black',s=200,alpha=0.2)
filter_Pt111_atoms = abs(Pt111.positions[:,1]-Pt111.get_center_of_mass()[1])<1.
plt.scatter(*Pt111.positions[filter_Pt111_atoms][:,::2].T,c='grey',edgecolor='black',s=200)

filter_data = system_2D_contactspace_top.data[np.abs(system_2D_contactspace_top.data['y'].values-system_2D.atoms.get_center_of_mass()[1]) < 0.1]
plt.scatter(filter_data[['x']].values,filter_data[['z']].values,s=1,color=default_colors[0],alpha=0.5)
filter_data = system_2D_contactspace_bottom.data[np.abs(system_2D_contactspace_bottom.data['y'].values-system_2D.atoms.get_center_of_mass()[1]) < 0.1]
plt.scatter(filter_data[['x']].values,filter_data[['z']].values,s=1,color=default_colors[1],alpha=0.5)

plt.title(r'$\left|\nabla s(r)\right|$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(system_2D.grid.cell[0,0])+1, 2)))
plt.yticks(list(range(0, int(system_2D.grid.cell[2,2])+1, 2)))
plt.gca().set_aspect('equal')
plt.xlim(0,system_2D.grid.cell[0,0]-0.16)
plt.ylim(0,system_2D.grid.cell[2,2])
plt.show()

### Build Ionic (Soft-Sphere) Boundary

In [ ]:
from mapsy.boundaries import IonicBoundary
ionic_boundary = IonicBoundary(mode = 'muff', grid=contact_space_grid, alpha = 1.12, softness = 1, system = system_2D, label='test')
ionic_boundary.update()

In [ ]:
xxx,yyy,zzz = ionic_boundary.switch.tocontour(system_2D.atoms.get_center_of_mass(),1)
contour = plt.contour(xxx,yyy,zzz,levels=[0.01,0.2,0.4,0.6,0.8,0.99],colors='black')
plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*Pt111.positions[:,::2].T,c='grey',edgecolor='black',s=200,alpha=0.2)
filter_Pt111_atoms = abs(Pt111.positions[:,1]-Pt111.get_center_of_mass()[1])<1.
plt.scatter(*Pt111.positions[filter_Pt111_atoms][:,::2].T,c='grey',edgecolor='black',s=200)

plt.title(r'$s(r)$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(system_2D.grid.cell[0,0])+1, 2)))
plt.yticks(list(range(0, int(system_2D.grid.cell[2,2])+1, 2)))
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "ionic", "radii" : "muff", "alpha": 1.12, "spread": 1, "cutoff": 100, "threshold": 0.4, 'side':1})
system_2D_contactspace_top = ContactSpaceGenerator(contactspacesettings).generate(system_2D)
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "ionic", "radii" : "muff", "alpha": 1.12, "spread": 1, "cutoff": 100, "threshold": 0.4, 'side':-1})
system_2D_contactspace_bottom = ContactSpaceGenerator(contactspacesettings).generate(system_2D)

In [ ]:
xxx,yyy,zzz = ionic_boundary.gradient.modulus.tocontour(system_2D.atoms.get_center_of_mass(),1)
contour = plt.contour(xxx,yyy,zzz,levels=[0.1,0.4,0.8,1.2],colors='black')
#plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
#
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*Pt111.positions[:,::2].T,c='grey',edgecolor='black',s=200,alpha=0.2)
filter_Pt111_atoms = abs(Pt111.positions[:,1]-Pt111.get_center_of_mass()[1])<1.
plt.scatter(*Pt111.positions[filter_Pt111_atoms][:,::2].T,c='grey',edgecolor='black',s=200)

filter_data = system_2D_contactspace_top.data[np.abs(system_2D_contactspace_top.data['y'].values-system_2D.atoms.get_center_of_mass()[1]) < 0.1]
plt.scatter(filter_data[['x']].values,filter_data[['z']].values,s=1,color=default_colors[0],alpha=0.5)
filter_data = system_2D_contactspace_bottom.data[np.abs(system_2D_contactspace_bottom.data['y'].values-system_2D.atoms.get_center_of_mass()[1]) < 0.1]
plt.scatter(filter_data[['x']].values,filter_data[['z']].values,s=1,color=default_colors[1],alpha=0.5)

plt.title(r'$\left|\nabla s(r)\right|$'+'\n')
plt.xlabel('X (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(system_2D.grid.cell[0,0])+1, 2)))
plt.yticks(list(range(0, int(system_2D.grid.cell[2,2])+1, 2)))
plt.gca().set_aspect('equal')
plt.xlim(0,system_2D.grid.cell[0,0]-0.16)
plt.ylim(0,system_2D.grid.cell[2,2])
plt.show()

## Contact Space for a Pt Cluster (0D)

### Generate a Pt Nanocluster

In [ ]:
from ase.cluster.cubic import FaceCenteredCubic
surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)]
layers = [5, 8, 5]
lc = 3.94000
PtCluster = FaceCenteredCubic('Pt', surfaces, layers, latticeconstant=lc)
PtCluster.cell = 30 * np.eye(3)
PtCluster.positions += [15, 15, 15]

In [ ]:
from mapsy.data import Grid
PtCluster_grid: Grid = Grid(cell=PtCluster.cell)

In [ ]:
from mapsy.data import System
cluster: System = System(PtCluster_grid, PtCluster, dimension=0, axis=0)

### Build Ionic Boundary

In [ ]:
from mapsy.utils import setscalars
cutoff = 40 # reciprocal space cutoff in Ry (same convetion as QE)
scalars = setscalars(cluster.grid.cell,cutoff)
contact_space_grid = Grid(scalars=scalars,cell=cluster.grid.cell)

In [ ]:
from mapsy.boundaries import IonicBoundary
ionic_boundary = IonicBoundary(mode = 'muff', grid=contact_space_grid, alpha = 1.2, softness = 0.8, system = cluster, label='test')
ionic_boundary.update()

In [ ]:
xxx,yyy,zzz = ionic_boundary.switch.tocontour(cluster.atoms.get_center_of_mass(),0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.01,0.2,0.4,0.6,0.8,0.99],colors='black')
plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*PtCluster.positions[:,::2].T,c='grey',edgecolor='black',s=200,alpha=0.05)
filter_PtCluster_atoms = abs(PtCluster.positions[:,1]-PtCluster.get_center_of_mass()[1])<1.
plt.scatter(*PtCluster.positions[filter_PtCluster_atoms][:,::2].T,c='grey',edgecolor='black',s=200)

plt.title(r'$s(r)$'+'\n'+'X = '+str(cluster.atoms.get_center_of_mass()[0])+' Å')
plt.xlabel('Y (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(cluster.grid.cell[0,0])+1, 5)))
plt.yticks(list(range(0, int(cluster.grid.cell[2,2])+1, 5)))
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
xxx,yyy,zzz = ionic_boundary.switch.tocontour(cluster.atoms.positions[0,:],0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.01,0.2,0.4,0.6,0.8,0.99],colors='black')
plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*PtCluster.positions[:,1:].T,c='grey',edgecolor='black',s=200,alpha=0.05)
filter_PtCluster_atoms = abs(PtCluster.positions[:,0]-cluster.atoms.positions[0,0])<1.
plt.scatter(*PtCluster.positions[filter_PtCluster_atoms][:,1:].T,c='grey',edgecolor='black',s=200)

plt.title(r'$s(r)$'+'\n'+f'X = {cluster.atoms.positions[0,0]:4.2f} Å')
plt.xlabel('Y (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(cluster.grid.cell[1,1])+1, 5)))
plt.yticks(list(range(0, int(cluster.grid.cell[2,2])+1, 5)))
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "ionic", "cutoff": 40, "alpha": 1.2, "spread" : 0.8, "threshold": 0.5})
cluster_contactspace = ContactSpaceGenerator(contactspacesettings).generate(cluster)

In [ ]:
xxx,yyy,zzz = ionic_boundary.gradient.modulus.tocontour(cluster.atoms.get_center_of_mass(),0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.001,0.5,0.8,1.2],colors='black')
#plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
#
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*PtCluster.positions[:,::2].T,c='grey',edgecolor='black',s=200,alpha=0.05)
filter_PtCluster_atoms = abs(PtCluster.positions[:,1]-PtCluster.get_center_of_mass()[1])<1.
plt.scatter(*PtCluster.positions[filter_PtCluster_atoms][:,::2].T,c='grey',edgecolor='black',s=200)

filter_data = cluster_contactspace.data[np.abs(cluster_contactspace.data['y'].values-cluster.atoms.get_center_of_mass()[1]) < 0.1]
plt.scatter(filter_data[['x']].values,filter_data[['z']].values,s=1,color=default_colors[0],alpha=0.5)

plt.title(r'$\left|\nabla s(r)\right|$'+'\n'+'X = '+str(cluster.atoms.get_center_of_mass()[0])+' Å')
plt.xlabel('X (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(cluster.grid.cell[0,0])+1, 5)))
plt.yticks(list(range(0, int(cluster.grid.cell[2,2])+1, 5)))
plt.gca().set_aspect('equal')
plt.xlim(0,cluster.grid.cell[0,0]-0.16)
plt.ylim(0,cluster.grid.cell[2,2])
plt.show()

In [ ]:
xxx,yyy,zzz = ionic_boundary.gradient.modulus.tocontour(cluster.atoms.positions[0,:],0)
contour = plt.contour(xxx,yyy,zzz,levels=[0.001,0.5,0.8,1.2],colors='black')
#plt.contourf(xxx,yyy,zzz,levels=200,cmap='Blues')
# Add labels to contour lines
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
#
plt.clabel(contour, inline=True, fontsize=10)
plt.scatter(*PtCluster.positions[:,::2].T,c='grey',edgecolor='black',s=200,alpha=0.05)
filter_PtCluster_atoms = abs(PtCluster.positions[:,1]-cluster.atoms.positions[0,0])<1.
plt.scatter(*PtCluster.positions[filter_PtCluster_atoms][:,::2].T,c='grey',edgecolor='black',s=200)

filter_data = cluster_contactspace.data[np.abs(cluster_contactspace.data['y'].values-cluster.atoms.positions[0,0]) < 0.1]
plt.scatter(filter_data[['x']].values,filter_data[['z']].values,s=1,color=default_colors[0],alpha=0.5)

plt.title(r'$\left|\nabla s(r)\right|$'+'\n'+f'X = {cluster.atoms.positions[0,0]:4.2f} Å')
plt.xlabel('X (Å)')
plt.ylabel('Z (Å)')
plt.xticks(list(range(0, int(cluster.grid.cell[0,0])+1, 5)))
plt.yticks(list(range(0, int(cluster.grid.cell[2,2])+1, 5)))
plt.gca().set_aspect('equal')
plt.xlim(0,cluster.grid.cell[0,0]-0.16)
plt.ylim(0,cluster.grid.cell[2,2])
plt.show()